In [1]:
import requests
from bs4 import BeautifulSoup
import json

from dask.array.reshape import contract_tuple

gs25_url = "https://gs25.gsretail.com/gscvs/ko/store-services/locations#;"

In [2]:
payload= {"pageNum" : "1",
"pageSize" : "100000",
"searchShopName" : "",
"searchSido" : "",
"searchGugun" : "",
"searchDong" : "",
"searchType" : "",
"searchTypeService" : "0",
"searchTypeToto" : "0",
"searchTypeCafe25" : "0",
"searchTypeInstant" : "0",
"searchTypeDrug" : "0",
"searchTypeSelf25" : "0",
"searchTypePost" : "0",
"searchTypeATM" : "0",
"searchTypeWithdrawal" : "0",
"searchTypeTaxrefund" : "0",
"searchTypeSmartAtm" : "0",
"searchTypeSelfCookingUtensils" : "0",
"searchTypeDeliveryService" : "0",
"searchTypeParcelService" : "0",
"searchTypePotatoes" : "0",
"searchTypeCardiacDefi" : "0",
"searchTypeFishShapedBun" : "0",
"searchTypeWine25" : "0",
"searchTypeGoPizza" : "0",
"searchTypeSpiritWine" : "0",
"searchTypeFreshGanghw" : "0",
"searchTypeMusinsa" : "0",
"searchTypePosa" : "0",}
header = {
    'host' : 'gs25.gsretail.com',
    'origin' : 'https://gs25.gsretail.com',
    'referer' : 'https://gs25.gsretail.com/gscvs/ko/store-services/locations'
}

In [3]:
url = "https://gs25.gsretail.com/gscvs/ko/store-services/locationList?CSRFToken={}"
with requests.Session() as s:
    r = s.get(gs25_url)
    bs = BeautifulSoup(r.text)
    csrf = bs.find("form", id='CSRFForm').find("input")['value']
    r2 = s.post(url.format(csrf), data=payload, headers=header)

In [4]:
ress = json.loads(r2.json())['results']
len(ress)

17784

In [5]:
import pandas as pd
df = pd.DataFrame(ress)
df.iloc[:, 1:].duplicated().sum()

0

In [7]:
import pymysql
conn = pymysql.connect(host='127.0.0.1', user='play', passwd='123', database='sk17', port=3306)
cur = conn.cursor()

In [20]:
table_name1 = 'gs25'
table_name2 = 'gs25_service'
insert_q1 = f"INSERT IGNORE INTO {table_name1} VALUES (%s, %s, %s, %s, %s)"
insert_q2 = f"INSERT IGNORE INTO {table_name2} VALUES (%s, %s)"
create_q1 = f"""create table {table_name1}(
    shopCode varchar(20) not null,
    longs float default null,
    address varchar(100) default null,
    shopName varchar(100) default null,
    lat float default null,
    primary key (shopCode)
)"""
create_q2 = f"""create table {table_name2} (
    shopCode varchar(20) not null,
    offeringService varchar(50) not null,
    primary key(shopCode, offeringService),
    FOREIGN KEY (shopCode) REFERENCES gs25(shopCode)
)"""
def create_table(tname,tquary):
    cur.execute('show tables')
    tables = [row[0] for row in cur.fetchall()]
    if tname in tables:
        cur.execute(f'drop table {tname}')
    cur.execute(tquary)

In [ ]:
create_table(table_name1,create_q1)

In [18]:
create_table(table_name2,create_q2)

In [14]:
for row in ress:
    print(row)
    break;

{'offeringService': ['toto', 'cafe25', 'post', 'withdrawal', 'parcel_service'], 'shopCode': 'V3414', 'longs': '37.51826292', 'address': '경기 부천시 오정구 석천로345,  300동 114호(삼정동 365)', 'shopName': 'GS252테크노파크점', 'lat': '126.76388845'}


In [15]:
for res in ress:
    row = ['']*5
    if 'shopCode' in res:
        row[0] = res['shopCode']
    if 'longs' in res:
        row[1] = res['longs']
    if 'address' in res:
        row[2] = res['address']
    if 'shopName' in res:
        row[3] = res['shopName']
    if 'lat' in res:
        row[4] = res['lat']
    cur.execute(insert_q1, row)

In [16]:
conn.commit()

In [21]:
for res in ress:
    row = ['']*2
    if 'shopCode' not in res:
        continue
    if 'offeringService' not in res:
        continue
    row[0] = res['shopCode']
    for service in res['offeringService']:
        row[1] = service
        cur.execute(insert_q2, row)

In [22]:
conn.commit()